In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import theano
pd.options.mode.chained_assignment = None

# Let us load our data file

In [2]:

full_df = pd.read_csv("twcs.csv", nrows=5000)

In [3]:
df = full_df[["text"]]
df["text"] = df["text"].astype(str) #declaring the contents in the col --
#--> "text" as a string variable.

full_df.head() # lets see what our rows and cols look like. 

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


# In order for us to model language and predict sarcasm, we must first wrangle and clean our data.

### We will first start with lower casing our "text". This will treat the string 'text', 'Text', 'TEXT' homogeneously. 

In [4]:
df["text_lower"] = df["text"].str.lower()
df.head()

,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.
